In [ ]:
import pandas as pd
from scipy.spatial.distance import euclidean, cityblock, cosine, correlation
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt

EDA:

In [ ]:
data=pd.read_csv("u.data", sep='\t', names=["user id", "item id", "rating", "timestamp"])
data=data.drop(columns=["timestamp"])
data

In [ ]:
sns.distplot(data.groupby("item id")['rating'].mean())

In [ ]:
median=data.groupby("item id")['rating'].mean().sort_values(ascending=False).median()
median

In [ ]:
info=pd.read_csv("u.info", sep='|')
info

In [ ]:
item=pd.read_csv("u.item", sep='|', names=["movie id","movie title", "release date", "video release date",
            "IMDb URL","unknown","Action","Adventure","Animation","Children's","Comedy","Crime","Documentary","Drama","Fantasy",
            "Film-Noir","Horror","Musical","Mystery","Romance","Sci-Fi","Thriller","War","Western"],encoding='latin-1')
item

In [ ]:
k=data.groupby("item id")['rating'].mean()
kind=["Action","Adventure","Animation","Children's","Comedy","Crime","Documentary","Drama","Fantasy",
            "Film-Noir","Horror","Musical","Mystery","Romance","Sci-Fi","Thriller","War","Western"]
rating=[k[item[item[i]==1]['movie id']].mean() for i in kind]
plt.figure(figsize=(15,5))
sns.barplot(x=kind,y=rating)
plt.show()

In [ ]:
genre=pd.read_csv("u.genre", sep='|', names=["Genre","#"])
genre

In [ ]:
user=pd.read_csv("u.user", sep='|', names=["user id","age", "gender", "occupation","zip code"])
user

In [ ]:
occupation=pd.read_csv("u.occupation", sep='|')
occupation

In [ ]:
data["user id"].value_counts()
# each user has rated at least 20 movies, no need to clean

In [ ]:
MoviePerUser=data["item id"].value_counts()
MoviePerUser

In [ ]:
item = item.set_index('movie id')
item

In [ ]:
movie_genre = item.drop(columns=['movie title', 'release date', 'video release date', 'IMDb URL'])
movie_genre

In [ ]:
movie_name = item.drop(item.iloc[:,1:], axis=1)
movie_name

Content-based

In [ ]:
def movie_recommender(distance_method, movie_id, N=1682):
    df_distance = pd.DataFrame(data=movie_genre.index)
    df_distance = df_distance[movie_genre.index != movie_id]
    df_distance['distance'] = df_distance['movie id'].apply(lambda x: distance_method(movie_genre.loc[x],movie_genre.loc[movie_id]))
    df_distance.sort_values(by='distance',inplace=True)
    return(df_distance.head(N))

In [ ]:
df_distance1=movie_recommender(euclidean, 1681)
df_distance1

In [ ]:
closestdis1=df_distance1[df_distance1['distance']==0]['movie id']

In [ ]:
movie_genre.loc[closestdis1,:]

In [ ]:
neardis1=df_distance1[df_distance1['distance']!=1]['movie id']
movie_genre.loc[neardis1,:]

In [ ]:
df_distance2=movie_recommender(cosine, 1681)
df_distance2

In [ ]:
closestdis2=df_distance2[df_distance2['distance']==0]['movie id']

In [ ]:
movie_genre.loc[closestdis2,:]

In [ ]:
neardis2=df_distance2[df_distance2['distance']!=1]['movie id']
movie_genre.loc[neardis2,:]

In [ ]:
df_distance3=movie_recommender(correlation, 1681)
df_distance3

In [ ]:
closestdis3=df_distance3[df_distance3['distance']==0]['movie id']

In [ ]:
movie_genre.loc[closestdis3,:]

In [ ]:
neardis3=df_distance3[df_distance3['distance']!=1]['movie id']
movie_genre.loc[neardis3,:].iloc[250:261]

In [ ]:
df_distance2.reset_index(inplace=True)
df_distance2

In [ ]:
cnt=0
result=[]
for i in range(len(df_distance2['distance'])-1):
    if df_distance2['distance'][i+1]==df_distance2['distance'][i]:
        result.append(cnt)
    else:
        cnt+=1
        result.append(cnt)
result.append(cnt)
result

In [ ]:
df_distance3.reset_index(inplace=True)

In [ ]:
cnt=0
result2=[]
for i in range(len(df_distance3['distance'])-1):
    if df_distance3['distance'][i+1]==df_distance3['distance'][i]:
        result2.append(cnt)
    else:
        cnt+=1
        result2.append(cnt)
result2.append(cnt)
result2

In [ ]:
df_distance2['result']=result

In [ ]:
df_distance2['result2']=result2
df_distance2

In [ ]:
df_distance2['result']!=df_distance2['result2']

In [ ]:
movie_recommender(correlation, 1)[:5]

collaborative user-user

In [ ]:
sum=0
for i in range(4):
    ratenum=MoviePerUser[MoviePerUser==i].sum()
    sum+=ratenum
    print ("rate by {} user(s): {} movies".format(i,ratenum))
print(sum)

In [ ]:
data = data[data["item id"].isin(MoviePerUser[MoviePerUser>3].index)]
data
# only keep movies have been rated by more than 5 users

In [ ]:
# create the user-item-rating matrix
UserMovieRatingMatrix=pd.pivot_table(data, values='rating', index=['user id'], columns=['item id'])
UserMovieRatingMatrix.head()

In [ ]:
from scipy.spatial.distance import hamming 
# lets write a function to compute k nearest neighbours of active user

def nearestneighbours(user,K):
    # create a user df that contains all users except active user
    allUsers = pd.DataFrame(UserMovieRatingMatrix.index)
    allUsers = allUsers[allUsers["user id"]!=user]
    # Add a column to this df which contains distance of active user to each user
    allUsers["distance"] = allUsers["user id"].apply(lambda x: hamming(UserMovieRatingMatrix.loc[user],UserMovieRatingMatrix.loc[x]))
    KnearestUsers = allUsers.sort_values(["distance"],ascending=True)["user id"][:K]
    return KnearestUsers


In [ ]:
nearestneighbours(5,2) #allUsers.sort_values(["distance"],ascending=True)

In [ ]:
import numpy as np
def topN(user, N=3, K=10):
    KnearestUsers = nearestneighbours(user,K)
    # get the ratings given by nearest neighbours
    NNRatings = UserMovieRatingMatrix[UserMovieRatingMatrix.index.isin(KnearestUsers)]
    # Find the average rating of each movie rated by the k nearest neighbours
    avgRating = NNRatings.apply(np.nanmean).dropna()
    # drop the movies already watched by active user
    MoviesWatched = UserMovieRatingMatrix.loc[user].dropna().index
    avgRating = avgRating[~avgRating.index.isin(MoviesWatched)] # ~ means NOT
    topMovies = avgRating.sort_values(ascending=False).index[:N]
    for index in topMovies:
        ans= print("Movie Recommended: ", item.at[index,"movie title"])
    return ans


In [ ]:
topN(3, N=5, K=10)

collaborative item-item

In [ ]:
sum=0
for i in range(4):
    ratenum=MoviePerUser[MoviePerUser==i].sum()
    sum+=ratenum
    print ("rate by {} user(s): {} movies".format(i,ratenum))
print(sum)

In [ ]:
data = data[data["item id"].isin(MoviePerUser[MoviePerUser>3].index)]
data
# only keep movies have been rated by more than 5 users

In [ ]:
# create the user-item-rating matrix
ItemMovieRatingMatrix=pd.pivot_table(data, values='rating', index=['item id'], columns=['user id'])
ItemMovieRatingMatrix.head()

In [ ]:
from scipy.spatial.distance import hamming 
# lets write a function to compute k nearest neighbours of active user

def nearestneighbours(item,K):
    # create a user df that contains all users except active user
    allItems = pd.DataFrame(ItemMovieRatingMatrix.index)
    allItems = allItems[allItems["item id"]!=item]
    # Add a column to this df which contains distance of active user to each user
    allItems["distance"] = allItems["item id"].apply(lambda x: hamming(ItemMovieRatingMatrix.loc[item],ItemMovieRatingMatrix.loc[x]))
    KnearestUsers = allItems.sort_values(["distance"],ascending=True)["item id"][:K]
    return KnearestUsers


In [ ]:
import numpy as np
def topN(item, N=3, K=10):
    KnearestUsers = nearestneighbours(item,K)
    # get the ratings given by nearest neighbours
    NNRatings = ItemMovieRatingMatrix[ItemMovieRatingMatrix.index.isin(KnearestUsers)]
    # Find the average rating of each movie rated by the k nearest neighbours
    avgRating = NNRatings.apply(np.nanmean).dropna()
    # drop the movies already watched by active user
    Itemrated = ItemMovieRatingMatrix.loc[item].dropna().index
    avgRating = avgRating[~avgRating.index.isin(Itemrated)] # ~ means NOT
    topMovies = avgRating.sort_values(ascending=False).index[:N]
    for index in topMovies:
        ans= print("Movie Recommended: ", Item.at[index,"movie title"])
    return ans


In [ ]:
topN(3, N=5, K=10)

Matrix factorization

In [ ]:
UserMovieRatingMatrix=pd.pivot_table(data, values='rating', index=['user id'], columns=['item id'])
UserMovieRatingMatrix

In [ ]:
def SGD(data):
    '''Learn the vectors p_u and q_i with SGD.
       data is a dataset containing all ratings + some useful info (e.g. number
       of items/users).
    '''

    n_factors = 19  # number of factors
    alpha = .01  # learning rate
    n_epochs = 10  # number of iteration of the SGD procedure

    # Randomly initialize the user and item factors.
    p = np.random.normal(0, .1, (943, n_factors))
    q = np.random.normal(0, .1, (1682, n_factors))

    # Optimization procedure
    for _ in range(n_epochs):
        for u in range(943):
            for i in range(1682):
                if np.isnan(data.iloc[u,i])==False:
                    err = data.iloc[u,i] - np.dot(p[u], q[i])
                    # Update vectors p_u and q_i
                    p[u] += alpha * err * q[i]
                    q[i] += alpha * err * p[u]
    return p,q

In [ ]:
p,q=SGD(UserMovieRatingMatrix)

In [ ]:
p

In [ ]:
q

In [ ]:
cnt=0
sm=0
for i in range(943):
    for j in range(1682):
        if np.isnan(UserMovieRatingMatrix.iloc[i,j])==False:
            sm+=(UserMovieRatingMatrix.iloc[i,j]-np.dot(p[i],q[j]))**2
            cnt+=1
accuracy=(sm/cnt)**(.5)
accuracy

In [ ]:
p[0]

In [ ]:
predictedrating=np.matmul(p,q.transpose())
predictedrating

In [ ]:
predictedrating[940,0]

In [ ]:
UserMovieRatingMatrix